In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from collections import OrderedDict
WORKING_DIR = '.'

dataset_educ = pd.read_json(f'{WORKING_DIR}/Dataset_educ_1.1.json', orient = 'index')
dataset_div = pd.read_json(f'{WORKING_DIR}/Dataset_div2_final.json', orient = 'index')

In [42]:
import re
from sklearn.metrics import classification_report
import os
import difflib

all_labels = ['SolutionExp', 'HintExp', 'AlgoExp','TimeComExp', 'FixingExp', 'TestExp', 'QuestionExp', 'SubQuestion', 'TestQuestion', 'Irrelevant']

def print_result(round_type):
    predicted_labels = {}
    truth_labels = {}

    for root, folders, files in os.walk(f'prompt_generation/{round_type}'):
        if(len(files) > 0):
            round_id = root.split('\\')[-1]
            for f in files:
                if(f.endswith('output.txt')):
                    with open(f"{root}/{f}", "r") as fp:
                        output_text = fp.read()
                    with open(f"chatgpt_predictions/no_tree/{round_type}/{round_id}/{f.split('_')[0] + '_prediction.txt'}", "r") as fp:
                        prediction_text = fp.read()
                
                    for comment in output_text.split('[<<<new_com>>>]')[:-1]:
                        comment_id = re.findall(r'\(+-*[0-9]+,[\s]\(*([0-9]+)\)+', comment)[0]
                        label = re.findall('|'.join(all_labels), comment)[0]

                        truth_labels[comment_id] = label

                    for comment in prediction_text.split('[<<<new_com>>>]')[:-1]:
                        comment_id = re.findall(r'\(+-*[0-9]+,[\s]\(*([0-9]+)\)+', comment)[0]
                        label = comment.split('<<===>>')[1]
                        label = difflib.get_close_matches(label, all_labels, n=1, cutoff=0)
                        predicted_labels[comment_id] = label
print_result('educ/test')
# predicted_labels = ['Relevant' if l!= 'Irrelevant' else l for l in predicted_labels]
# truth_labels = ['Relevant' if l!= 'Irrelevant' else l for l in truth_labels]
# print(classification_report(truth_labels, predicted_labels))

['AlgoExp']
['Irrelevant']
['SolutionExp']
['AlgoExp']
['SolutionExp']
['SolutionExp']
['AlgoExp']
['HintExp']
['Irrelevant']
['SolutionExp']
['Irrelevant']
['TestExp']
['Irrelevant']
['QuestionExp']
['SubQuestion']
['HintExp']
['QuestionExp']
['FixingExp']
['FixingExp']
['Irrelevant']
['SubQuestion']
['SubQuestion']
['HintExp']
['QuestionExp']
['SolutionExp']
['SolutionExp']
['HintExp']
['SolutionExp']
['QuestionExp']
['AlgoExp']
['Irrelevant']
['TestExp']
['SolutionExp']
['Irrelevant']
['TestQuestion']
['TestExp']
['TestExp']
['FixingExp']
['FixingExp']
['AlgoExp']
['SolutionExp']
['SolutionExp']
['FixingExp']
['QuestionExp']
['AlgoExp']
['SolutionExp']
['SolutionExp']
['SolutionExp']
['TimeComExp']
['TimeComExp']
['Irrelevant']
['SolutionExp']
['Irrelevant']
['Irrelevant']
['Irrelevant']
['QuestionExp']
['SubQuestion']
['FixingExp']
['Irrelevant']
['AlgoExp']
['HintExp']
['QuestionExp']
['SolutionExp']
['Irrelevant']
['SubQuestion']
['SolutionExp']
['TestQuestion']
['Irrelevant']
['

In [ ]:
for idx, round in enumerate(div_groups):
    round_df = round[1]

    for _, comment in round_df.iterrows():
        truth_labels[comment.id] = comment.label

    chatgpt_conversation = ''

    with open(f'conversations/div/{idx}.txt', 'r', encoding='utf-8', errors='ignore') as fp:
        chatgpt_conversation = fp.read()
        chatgpt_conversation = chatgpt_conversation.split('|^_^|')
    
    #skip first comment
    chatgpt_conversation = chatgpt_conversation[1:]

    for comment in chatgpt_conversation:
        if("prompt: %rule%" in comment):
           response = comment.split("response:")[1]
           
           found = re.findall(r'\(+-*[0-9]+,[\s\(]+([0-9]+)\)+ => (.*)', response)

           for found_id, found_label in found:
               predicted_labels[int(found_id)] = found_label

print(f"Total comments: {len(truth_labels)}")
print(f"Missing comments: {len(set(truth_labels.keys()) - set(predicted_labels.keys()))}")
print(f"Extra comments: {len(set(predicted_labels.keys()) - set(truth_labels.keys()))}")

common_comment_ids = list(set(truth_labels.keys()).intersection(set(predicted_labels.keys())))

common_predicted_labels = [predicted_labels[id] if predicted_labels[id] == 'Irrelevant' else 'Relevant' for id in common_comment_ids]
common_truth_labels = [truth_labels[id] if truth_labels[id] == 'Irrelevant' else 'Relevant' for id in common_comment_ids]

print(classification_report(common_truth_labels, common_predicted_labels))

Total comments: 664
Missing comments: 26
Extra comments: 0
              precision    recall  f1-score   support

  Irrelevant       0.97      0.31      0.47       452
    Relevant       0.37      0.98      0.54       186

    accuracy                           0.51       638
   macro avg       0.67      0.65      0.51       638
weighted avg       0.80      0.51      0.49       638

